## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [5]:
# download the data:
!wget "https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?raw=1" -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2020-09-17 17:17:36--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?raw=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/obaitrix9jyu84r/quora.txt [following]
--2020-09-17 17:17:37--  https://www.dropbox.com/s/raw/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5a2ab96fa0ef11673ba3d7dc3f.dl.dropboxusercontent.com/cd/0/inline/A_kSMG6co0Vd--Am5uLf3PFt79OaRXbDMK5tAnZu4BPFlguYZE6EoBrOCPbvEckpapwBcsGMSkUCv3sKeaFclRI1r_JEHnGeNt1sBO6gUNdvHQ/file# [following]
--2020-09-17 17:17:37--  https://uc5a2ab96fa0ef11673ba3d7dc3f.dl.dropboxusercontent.com/cd/0/inline/A_kSMG6co0Vd--Am5uLf3PFt79OaRXbDMK5tAnZu4BPFlguYZE6EoBrOCPbvEckpapwBcsGMSkUCv3sKeaFclRI1r_JEHnGeNt1sBO6gUNdvHQ/file
Resolving uc5a2ab96fa0ef11673ba3d7dc3f.dl.dropb

In [6]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [7]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [10]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.
from tqdm import tqdm

data_tok = []
for line in tqdm(data):
    data_tok.append(tokenizer.tokenize(line.lower()))

100%|██████████| 537272/537272 [00:03<00:00, 149063.68it/s]


In [11]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

In [12]:
print([" ".join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [13]:
from gensim.models import Word2Vec

model = Word2Vec(
    data_tok,
    size=32,  # embedding vector size
    min_count=5,  # consider words that occured at least 5 times
    window=5,
).wv  # define context as a 5-word window around the target word

In [14]:
# now you can get word vectors !
model.get_vector("anything")

array([-2.7050958 ,  0.27297834,  1.6429273 ,  0.86581606,  0.7229109 ,
       -2.251627  ,  0.26172283,  2.797283  ,  1.6246026 , -1.2603924 ,
        1.2813293 ,  2.7649515 , -0.8560707 ,  1.0352638 , -1.3015975 ,
       -0.8157279 ,  1.8249562 , -2.8216007 ,  0.4044109 ,  3.7278137 ,
       -2.4503245 ,  2.1496563 , -4.2665415 , -0.666826  , -2.4066293 ,
       -1.553739  ,  0.74591994, -0.14650393, -0.82165223,  1.8008082 ,
        3.8903887 , -1.5470902 ], dtype=float32)

In [15]:
# or query similar words directly. Go play with it!
model.most_similar("bread")

[('rice', 0.9601035118103027),
 ('cheese', 0.9272956848144531),
 ('butter', 0.926301121711731),
 ('sauce', 0.9251174926757812),
 ('potatoes', 0.912155270576477),
 ('garlic', 0.9085032343864441),
 ('fruit', 0.9065208435058594),
 ('cream', 0.9059382081031799),
 ('grass', 0.9041897058486938),
 ('chocolate', 0.9020166397094727)]

In [16]:
model.most_similar("happiness")

[('horoscope', 0.7617946863174438),
 ('creativity', 0.7277348041534424),
 ('nature', 0.7026760578155518),
 ('destiny', 0.6925961375236511),
 ('sense', 0.6857423782348633),
 ('consciousness', 0.6839867830276489),
 ('anatomy', 0.6839781999588013),
 ('soul', 0.6810428500175476),
 ('success', 0.6801801323890686),
 ('karma', 0.6759414672851562)]

In [17]:
model.most_similar("money")

[('cash', 0.7915061116218567),
 ('bitcoin', 0.7216474413871765),
 ('customers', 0.7143117189407349),
 ('profit', 0.6978917121887207),
 ('permission', 0.6964873671531677),
 ('profits', 0.6905885934829712),
 ('investing', 0.6868938207626343),
 ('stock', 0.6837437152862549),
 ('paypal', 0.6771611571311951),
 ('taxes', 0.6736736297607422)]

In [18]:
model.most_similar("love")

[('dream', 0.7873575687408447),
 ('happy', 0.7751324772834778),
 ('feelings', 0.7524087429046631),
 ('talking', 0.7415167093276978),
 ('talk', 0.7396593689918518),
 ('hope', 0.7392035126686096),
 ('boyfriend', 0.7276818752288818),
 ('friends', 0.7263224720954895),
 ('girlfriend', 0.7239848971366882),
 ('friendship', 0.7156384587287903)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [19]:
import gensim.downloader as api

model = api.load("glove-twitter-100")

[==================================================] 100.0% 387.1/387.1MB downloaded


In [20]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.538511335849762),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233934879303),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

In [21]:
model.most_similar(positive=["startup", "time"], negative=["money"])

[('startups', 0.5783799290657043),
 ('networking', 0.5626072883605957),
 ('summit', 0.5593665838241577),
 ('tech', 0.5576900839805603),
 ('blog', 0.5496704578399658),
 ('discovery', 0.5468578338623047),
 ('innovation', 0.5414659976959229),
 ('start-up', 0.53978431224823),
 ('launch', 0.5379266738891602),
 ('meetup', 0.5309116840362549)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [22]:
words = sorted(
    model.vocab.keys(), key=lambda word: model.vocab[word].count, reverse=True
)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [23]:
# for each word, compute it's vector with model
word_vectors = np.asarray([model.get_vector(word) for word in words])

In [24]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [29]:
def normalize(vectors):
    return (vectors - vectors.mean(axis=0)) / vectors.std(axis=0)

In [25]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)

# and maybe MORE OF YOUR CODE here :)
word_vectors_pca = normalize(word_vectors_pca)

In [26]:
assert word_vectors_pca.shape == (
    len(word_vectors),
    2,
), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [27]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs
):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

Loading BokehJS ...

In [28]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [30]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = normalize(TSNE(n_components=2).fit_transform(word_vectors))

In [31]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color="green", token=words)

Figure(id='1100', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [36]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    vector = np.zeros([model.vector_size], dtype="float32")

    phrase_tokenized = tokenizer.tokenize(phrase.lower())
    total = 0

    for token in phrase_tokenized:
        if token in model.vocab:
            total += 1
            vector += model.get_vector(token)

    if total:
        vector /= total

    return vector

In [37]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(
    vector[::10],
    np.array(
        [
            0.31807372,
            -0.02558171,
            0.0933293,
            -0.1002182,
            -1.0278689,
            -0.16621883,
            0.05083408,
            0.17989802,
            1.3701859,
            0.08655966,
        ],
        dtype=np.float32,
    ),
)

In [40]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[:: len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.asarray([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [41]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [42]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (
    phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)
) / phrase_vectors_2d.std(axis=0)

In [43]:
draw_vectors(
    phrase_vectors_2d[:, 0],
    phrase_vectors_2d[:, 1],
    phrase=[phrase[:50] for phrase in chosen_phrases],
    radius=20,
)

Figure(id='1206', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [44]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [73]:
def compute_similarity(query_vector, ref_vector):
    numerator = np.dot(query_vector, ref_vector)
    denominator = np.linalg.norm(query_vector) * np.linalg.norm(ref_vector)
    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator != 0)


def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_vector = get_phrase_embedding(query)
    similarity = np.asarray(
        [compute_similarity(query_vector, ref_vector) for ref_vector in data_vectors]
    )

    ind = np.argpartition(similarity, -k)[-k:]
    ind = ind[np.flip(np.argsort(similarity[ind]))]

    return [data[idx] for idx in ind]

In [74]:
results = find_nearest(query="How do i enter the matrix?", k=10)

In [75]:
print("".join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == "How do I get to the dark web?\n"
assert results[3] == "What can I do to save the world?\n"

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [76]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [77]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

In [78]:
find_nearest(query="Believe in yourself", k=10)

['Can someone believe in the existence of God without believing in any religion?\n',
 'How can Buddhists believe in reincarnation without believing in the concept of a "self"?\n',
 'Why should you believe in yourself?\n',
 'People believe in God but does God believe in people?\n',
 'Do you believe in the existence of self?\n',
 'How can you start believing in yourself?\n',
 'Have you found yourself in live?\n',
 "Is it possible to fool yourself into believing something in the past didn't happen?\n",
 'How can I believe in myself?\n',
 'What is the best way to believe in yourself?\n']

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.